# Conten Based filtering using audio features, album_uri and artist_uri

In [1]:
import import_ipynb

In [2]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import random
import scipy.sparse as sps

from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
from category_encoders import TargetEncoder
from time import time

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

#own functions
from evaluation import DCG
from evaluation import nDCG
from evaluation import R_Precision


%matplotlib inline

importing Jupyter notebook from evaluation.ipynb
DCG = 0.5
IDCG = 1.0
nDCG = 0.5


# Data transformation, PCA and merging 

In [3]:
with open('../data-processed/full-data/track_descriptions.json') as json_file:
    D_desc = json.load(json_file)
    
D_desc['spotify:track:0UaMYEvWZi0ZqiDOoHU3YI']

['Lose Control (feat. Ciara & Fat Man Scoop)', 'Missy Elliott', 'The Cookbook']

In [4]:
with open('../data-processed/full-data/track_artist_album.json') as json_file:
    D_album_artist = json.load(json_file)
    
D_album_artist['spotify:track:0UaMYEvWZi0ZqiDOoHU3YI']

['spotify:artist:2wIVse2owClT7go1WT98tk',
 'spotify:album:6vV5UrXcfyQD1wu4Qo2I9K']

In [5]:
path = '../data-processed/full-data/audio-features-combined.csv'
data = pd.read_csv(path)

In [6]:
data = data.reset_index()
data = data.rename(columns = {'index':'track_id'})
data['artist_uri'] = data.apply(lambda x:D_album_artist[x.uri][0], axis = 1)
data['album_uri'] = data.apply(lambda x:D_album_artist[x.uri][1], axis = 1)

In [7]:
# data.head()

In [8]:
D_track_uri_to_id = data.groupby('uri')['track_id'].min().to_dict()
D_track_id_to_uri = data.groupby('track_id')['uri'].min().to_dict()

In [9]:
len(D_track_uri_to_id), len(D_track_id_to_uri)

(2262190, 2262190)

In [10]:
data.head()

,track_id,uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,artist_uri,album_uri
0,0,spotify:track:5OSaZxhNj307YpTh7Qp8Xy,0.716,0.421,7,-11.560,0,0.0275,0.649000,0.002070,0.0936,0.2070,99.078,378440,4,spotify:artist:1vSHzGHsVOCrgPSCmKNimP,spotify:album:0oA1WetYmmrxkyjxz92yJg
1,1,spotify:track:4zytmsfZ7HtdXj3VLECcu2,0.141,0.441,5,-12.893,1,0.0892,0.700000,0.687000,0.1280,0.0725,76.044,331000,4,spotify:artist:6gHYtzSvIIWvoUY2q2V3Rj,spotify:album:3c1gpGKfahKOFXtM7eIcFO
2,2,spotify:track:6lOWUS7iBVEw6ieJqxH17B,0.799,0.893,8,-5.496,1,0.0635,0.000018,0.914000,0.3620,0.5290,128.009,469397,4,spotify:artist:7kxOVclB0zQamtBR0syCrg,spotify:album:0mX83KZvGWqBOvI4nIZ23H
3,3,spotify:track:3OSL6hJ9DoRGwr9OSEVrRS,0.789,0.548,4,-7.167,1,0.0418,0.707000,0.000934,0.0616,0.5930,117.713,153893,4,spotify:artist:2sxmKe3CUrWnx7eoXMhOlW,spotify:album:4hXCM8vqLJnlFcuHoH3zVP
4,4,spotify:track:6x0bgGOKckFsesVf7yPWJq,0.728,0.670,3,-7.912,0,0.1020,0.157000,0.000797,0.1080,0.0647,119.963,196000,4,spotify:artist:6PyeXqjH8OMGnt1IOhWgrQ,spotify:album:1pjNAADvPDurRS42fqxN4k


In [11]:
##
genres = pd.read_csv('../data-processed/full-data/genres_by_artist.csv')

In [12]:
##
genres.head()

,artist_uri,21st century classical,432hz,8-bit,8d,a cappella,aarhus indie,aberdeen indie,abstract,abstract beats,...,zim hip hop,zim urban groove,zimdancehall,zither,zolo,zouglou,zouk,zouk riddim,zurich indie,zydeco
0,spotify:artist:0001ZVMPt41Vwzt1zsmuzp,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,spotify:artist:0001wHqxbF2YYRQxGdbyER,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,spotify:artist:000Dq0VqTZpxOP6jQMscVL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,spotify:artist:000spuc3oKgwYmfg5IE26s,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,spotify:artist:000xagx3GkcunHTFdB4ly0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
##
##data_genres = data.merge(genres, how='left', left_on='artist_uri', right_on='artist_uri')

In [14]:
# data_genres.shape

# PCA on genres

In [15]:
X_genres = genres.iloc[:,1:]

In [16]:
X_genres.shape

(122142, 4967)

In [17]:
X_genres.values

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [18]:
pca = PCA(n_components=1000)
pca = pca.fit(X_genres.values)
print(sum(pca.explained_variance_ratio_))

0.6643661373047783


In [19]:
X_genres_pca = pca.transform(X_genres)

In [20]:
genres_pca_df = pd.DataFrame(X_genres_pca)

In [21]:
genres_pca_df.shape

(122142, 1000)

In [22]:
genres_pca_df.shape, genres.shape

((122142, 1000), (122142, 4968))

In [23]:
genres_pca_df['artist_uri'] = genres['artist_uri']

In [24]:
new_cols = ['pca_genre_'+ str(el) for el in list(genres_pca_df.columns)]

In [25]:
genres_pca_df.columns = new_cols

In [26]:
genres_pca_df.head()

,pca_genre_0,pca_genre_1,pca_genre_2,pca_genre_3,pca_genre_4,pca_genre_5,pca_genre_6,pca_genre_7,pca_genre_8,pca_genre_9,...,pca_genre_991,pca_genre_992,pca_genre_993,pca_genre_994,pca_genre_995,pca_genre_996,pca_genre_997,pca_genre_998,pca_genre_999,pca_genre_artist_uri
0,-0.014280,-0.018220,0.000345,-0.002941,-0.004096,-0.003106,-0.011270,-0.014823,-0.002936,-0.004452,...,0.003238,-0.000925,-0.002489,0.001668,0.006311,0.006227,-0.002603,-0.003895,0.001850,spotify:artist:0001ZVMPt41Vwzt1zsmuzp
1,-0.015368,-0.019184,0.000772,-0.005146,-0.004233,-0.001388,-0.010450,-0.009148,-0.001779,-0.003654,...,0.003238,0.005511,0.002680,-0.000447,0.004783,0.007895,0.000365,-0.000590,-0.003640,spotify:artist:0001wHqxbF2YYRQxGdbyER
2,-0.018196,-0.000024,0.033606,-0.030348,0.004702,0.003801,-0.006495,-0.003460,-0.000118,-0.003463,...,-0.018287,0.001350,-0.022866,-0.001153,0.044612,-0.046865,-0.005675,-0.034892,-0.019184,spotify:artist:000Dq0VqTZpxOP6jQMscVL
3,-0.014593,-0.018131,0.000744,-0.004695,-0.003859,-0.001276,-0.009498,-0.008311,-0.001632,-0.003259,...,0.016836,-0.043090,-0.032063,-0.001364,-0.054358,-0.039975,-0.018503,-0.046657,-0.024826,spotify:artist:000spuc3oKgwYmfg5IE26s
4,-0.014374,-0.017789,0.000655,-0.004391,-0.003389,-0.001136,-0.009248,-0.007998,-0.001620,-0.003173,...,0.000211,0.003280,-0.003754,0.005967,-0.001261,-0.000395,0.001990,0.000547,-0.000139,spotify:artist:000xagx3GkcunHTFdB4ly0


In [27]:
genres_pca_df.to_csv('../data-processed/transformation-matrices/cb_genres_pca_df.csv', index = None)

# Plot data

In [28]:
# data.iloc[:,1:-1].hist(figsize=(14,9))
# uncomment
# plt.show()

In [29]:
# fig, ax = plt.subplots(ncols=4, nrows=3, figsize=(14,9))

# column = data.columns[2:14]

#uncomment
# for i in range(3):
#     for j in range(4):
#         sns.boxplot(data=data[column[i*4+j]], palette="Set1", ax=ax[i,j]).set_title(column[i*4+j])

# Standardize data 

In [30]:
X = data.iloc[:,2:13]

In [31]:
#column orders
data.iloc[:,2:13].columns

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo'],
      dtype='object')

In [32]:
scaler = StandardScaler()
transformer = scaler.fit(X)
X_transformed = transformer.transform(X)

In [33]:
# uncomment
# pd.DataFrame(X_transformed).hist(figsize=(14,9))
# plt.show()

# Target encode album_uri and artist_uri

In [34]:
X_transformed.shape

(2262190, 11)

In [35]:
df_X_transformed = pd.DataFrame(X_transformed, columns=data.iloc[:,2:13].columns)
# df_X_transformed.head()

In [36]:
df_X_transformed.shape

(2262190, 11)

In [37]:
df_X_transformed.columns

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo'],
      dtype='object')

# Encode album_uri

In [38]:
cols = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']

In [39]:
df_X_transformed['album_uri'] = data.album_uri
X = data['album_uri'].values.reshape(-1,1)

In [ ]:
enc = TargetEncoder()
for col in cols:
    y = df_X_transformed[col].values.reshape(-1,1)  
    df_X_transformed[f'album_uri_{col}_enc'] = enc.fit_transform(X, y)

In [ ]:
# df_X_transformed.head()

# Encode artist_uri

In [ ]:
cols = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']

In [ ]:
df_X_transformed['artist_uri'] = data.artist_uri
X = data['artist_uri'].values.reshape(-1,1)

In [ ]:
enc = TargetEncoder()
for col in cols:
    y = df_X_transformed[col].values.reshape(-1,1) 
    df_X_transformed[f'artist_uri_{col}_enc'] = enc.fit_transform(X, y)

In [ ]:
# df_X_transformed.head()

In [ ]:
#X_transformed = df_X_transformed.drop(columns=['artist_uri','album_uri']).to_numpy()

In [ ]:
df_X_transformed.shape

In [ ]:
df_X_transformed.head()

In [ ]:
df_X_transformed.to_csv('../data-processed/transformation-matrices/cb_df_X_transformed.csv', index = None)

# Add Genres PCA

In [ ]:
df_X_transformed.columns

In [ ]:
df_X_transformed

In [ ]:
df_merged = df_X_transformed.merge(genres_pca_df, how = 'left', left_on = 'artist_uri', right_on ='pca_genre_artist_uri' )

In [ ]:
df_merged.shape

In [ ]:
df_merged.head()

In [ ]:
#df_merged = df_merged.fillna(0)

In [ ]:
df_merged.to_csv('../data-processed/transformation-matrices/cb_df_merged.csv', index = None)

In [ ]:
X_transformed = df_merged.drop(columns=['artist_uri','album_uri']).to_numpy()